В данной работе проводится разведывательный анализ данных для предсказания рейтинга ресторона в TripAdvizor

В ходе работы будут выполнены следующие действия
1. Импорт библиотек, загрузка и первичный анализ набора данных
2. Анализ и обработка признаков из дата сета
3. Корреляционный анализ, проверка на выбросы
4. Подготовка модели машинного обучения, анализ ее работы



# 1. Импорт библиотек, загрузка и первычный анализ набора данных

In [343]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline
from itertools import combinations
from scipy.stats import ttest_ind
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
from collections import Counter
from string import punctuation
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from datetime import datetime
# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [344]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [345]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

Проводим загрузку данных в обучающую и тестовую выборку

In [346]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [347]:
df_train.info()

В датасете 10 столбцов и всего 40 000 строк. Три столбца Cuisine Style, Price range, Number of reviews имеют пропущенные значения, а также только эти столбцы числовые.

In [348]:
df_train.head(5)

In [349]:
df_test.info()

В тестовой выборке 10 000 строк, в 3 переменных пропущенные значениям

In [350]:
df_test.head(5)

In [351]:
sample_submission.head(5)

In [352]:
sample_submission.info()

In [353]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

Проверим наличие неуникальных значений в дата сете

In [354]:
df_train.nunique()

Признак Restaurant_id не является уникальным идектификатором ресторана, возможно из-за того что часть рестронов является сетевыми или франшизами. Из возможно полезного отметит что признак City содержит 31 город, Rating - 9 вариантов рейтинга, и Price Range - 3 возможных диапазона цены.

In [355]:
data.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [356]:
data.sample(15)

# 2. Анализ и обработка признаков из набора данных

Создадим списки для категориальных признаков, числовых и бинарных с типами наших значений, куда будем отправлять каждый из признаков после анализа

In [357]:
cat_cols = []  
num_cols = []  
bin_cols = []

Создадим функцию для анализа признаков

In [358]:
def column_info(col):
    print('Количество пропусков: {}\n'.format(col.isna().sum()))
    print('{}\n'.format(col.describe()))
    print('Распределение:\n{}\n'.format(col.value_counts()))

Создадим функцию проверяющую наличие выбросов через межквартильный размах

In [359]:
def ejection(column):
    IQR = data[column].quantile(0.75) - data[column].quantile(0.25)
    perc25 = data[column].quantile(0.25)
    perc75 = data[column].quantile(0.75)
    print("Границы выбросов:", perc25 - 1.5 * IQR, perc75 + 1.5 * IQR)

Создадим функцию очистки строки от любых символов и цифр

In [360]:
def clean_symbs(data):
    clean = re.sub(r"[^a-zA-Z_]", " ", data)
    return clean

Создадим функцию подсчитывающую количество слов в предложении

In [361]:
counter = Counter()
def count_words(sentence):
    global сounter
    for x in sentence:
        сounter[x] += 1
        

In [362]:
# функция проверяет, есть ли статистическая разница в распределении оценок по номинативным признакам, с помощью теста Стьюдента.
def get_stat_dif(data, column, score_column):

    cols = data_nom.loc[:, column].value_counts().index[:10]
    combinations_all = list(combinations(cols, 2))
    for comb in combinations_all:
        p_value = ttest_ind(data.loc[data.loc[:, column] == comb[0], score_column],
                            data.loc[data.loc[:, column] == comb[1], score_column]).pvalue
        if p_value <= 0.05/len(combinations_all):  # Учли поправку Бонферони
            print('Внимание! Найдены статистически значимые различия для переменных {} и {}'
                  .format(column, score_column))
            break
    else:
        print('Статистические различия для переменных {} и {} не найдены '
              .format(column, score_column))

# функция принимает датафрейм с переменными, два признака и показывает boxplot
def get_boxplot(data, column, score_column):
    fig, ax = plt.subplots(figsize=(10, 4))
    sns.boxplot(x=column, y='Rating', data=data.loc[data.loc[:, column].isin(data.loc[:, column].value_counts().index[:10])], ax=ax)
    plt.xticks(rotation=45)
    ax.set_title('Boxplot for ' + column)
    plt.show()

1. Поиск и анализ пропущенных значений. Вычислим процент максимального / минимального количества пропусков в стоках набора и визуализируем пропуски в данных для принятия решения об их обработке

In [363]:
misses_max = max(data.apply(lambda x: sum(
    x.isnull()), axis=1,))/data.shape[1]
misses_min = min(data.apply(lambda x: sum(
    x.isnull()), axis=1,))/data.shape[1]
print('Максимальное количество пропусков в строке:', round(misses_max*100), '%')
print('Минимальное количество пропусков в строке:', round(misses_min*100), '%')

In [364]:
clmn = data.columns[2:6]
sns.heatmap(data[clmn].isnull(), cmap=sns.color_palette("rocket"))

1. Пропущенные значения выделены светлыми полосами на графике и распределены равномерно, без явно выраженных кластеров. Максимальное количество пропусков в данных составляет 36%. Пропущенные значения заменим на наиболее часто встречающиеся.

Проведем очистку признаков и их преобразование к числовым значениям

In [365]:
# Значения ID  ресторана преобразуем в число
data.Restaurant_id = data.Restaurant_id.apply(lambda x: x[3:])
data.Restaurant_id = [int(x) for x in data.Restaurant_id]

# Значения ID ресторана в TripAdvisor преобразуем в числом
data.ID_TA = data.ID_TA.apply(lambda x: x[1:])
data.ID_TA = [int(x) for x in data.ID_TA]

# Удалим символы и знаки препинания в Cuisine Style
data['Cuisine Style'] = data['Cuisine Style'].astype(str).apply(lambda x: None if x.strip() == '' else x)
data['Cuisine Style'] = data['Cuisine Style'].apply(lambda x: x.replace("['", '').replace("', '", ',').replace("']", ''))

# Удалим символы и знаки препинания в Reviews
data['Reviews'] = data['Reviews'].astype(str).apply(lambda x: None if x.strip() == '' else x)
data['Reviews'] = data['Reviews'].apply(lambda x: x.replace("[['", '').replace("', '", '|').replace('], [', '|').replace("']]", ''))
data['Reviews'] = data['Reviews'].apply(lambda x: x.replace("'", ''))

# Удалим /Restaurant_Review-, -Reviews-, .html из URL_TA
data['URL_TA'] = data['URL_TA'].astype(str).apply(lambda x: None if x.strip() == '' else x)
data['URL_TA'] = data['URL_TA'].apply(lambda x: x.replace("/Restaurant_Review-", '').replace(".html", '').replace('-Reviews-', '|'))
data['URL_TA'] = data['URL_TA'].str.split('|', expand=True)

1. Обработка признака Restaraunt_ID

In [366]:
fig = px.histogram(data, x=data['Restaurant_id'])
fig.show()
data.corr()['Restaurant_id']


На графике отображается линейная зависимость - чем меньше значение идентификатора ресторона тем больше количество записей о нем в дата сете. 
Признак Restaurant_id имеет высокую прямую корреляцию с местом ресторона в рейтинге и слабую обратную корреляцию с оценкой ресторана клиентами Rating

2. Обработка признака City

In [367]:
column_info(data.City)

In [368]:
plt.figure(figsize=(14,6))

a = sns.countplot(x='City', 
                  data=data,
                 color='red')
a.set_title("Restaraunts", fontsize=20)
a.set_xlabel("City ", fontsize=20)
a.set_ylabel("Quantity", fontsize=20)
a.set_xticklabels(a.get_xticklabels(),rotation=45)
plt.show()


Большинство ресторанов расположены в столицах и крупных городах западной и центральной Европы, что вызвано количеством платежеспособного населения в этих городах и туристическими потоками

Сгруппируем рестораны по городам и рейтингу, ренкингу и количеству обзоров ресторанов

In [369]:
City_rating = data.groupby('City')['Rating'].median().sort_values()
City_rating = City_rating.reset_index()
City_rating.columns = ['Rating','City']
print(City_rating)

Практически у всех ресторанов медиана в рейтинге 4.0 

In [370]:
City_Ranking = data.groupby('City')['Ranking'].median().sort_values()
City_Ranking = City_Ranking.reset_index()
City_Ranking.columns = ['City','Ranking']
print(City_Ranking)

Города в которых больше всего ресторонов в занимают в среднем более высокие места в рейтинге ресторанов

Добавим население в городах, из открытых источников

In [371]:
City_population = {
    'London': 7556900,
    'Paris': 2138551,
    'Madrid': 3255944,
    'Barcelona': 1621537,
    'Berlin': 3426354,
    'Milan': 1236837,
    'Rome': 2318895,
    'Prague': 1165581,
    'Lisbon': 517802,
    'Vienna': 1691468,
    'Amsterdam': 741636,
    'Brussels': 1019022,
    'Hamburg': 1739117,
    'Munich': 1260391,
    'Lyon': 472317,
    'Stockholm': 1515017,
    'Budapest': 1741041,
    'Warsaw': 1702139,
    'Dublin': 1024027,
    'Copenhagen': 1153615,
    'Athens': 664046,
    'Edinburgh': 464990,
    'Zurich': 341730,
    'Oporto': 249633,
    'Geneva': 183981,
    'Krakow': 755050,
    'Oslo': 580000,
    'Helsinki': 558457,
    'Bratislava': 423737,
    'Luxembourg': 76684,
    'Ljubljana': 272220
}

data['City_Population'] = data['City'].map(City_population)

Создадим новый признак количество ресторанов в городе

In [372]:
Restaurants_count_list = data.groupby('City').ID_TA.count().to_dict()
data['Restaurants_Count'] = data['City'].map(Restaurants_count_list)

На основе населения городов и количества ресторанов в городах создадим новый признак

In [373]:
data['Population_Restaurants'] = (data['Restaurants_Count'] / data['City_Population'])

Добавим признаки в списки

In [374]:
num_cols.append('City_Population')
num_cols.append('Restaurants_Count')
num_cols.append('Population_Restaurants')

In [375]:
data.info()

3. Обработка признака Cusine style.
Подсчитаем количество пропущенных значений, посмотрим на наиболее часто встречающееся значение

In [376]:
column_info(data['Cuisine Style'])

Составим список всех кухонь, заменим пропуски на наиболее часто встречающиеся значения

In [377]:
 data.info()

In [378]:
cuisines = pd.DataFrame(data['Cuisine Style'].str.split(',').tolist()).stack().value_counts().head(60)
cuisines

In [379]:
data['Cuisine Style'] = data['Cuisine Style'].apply(lambda x: x.replace('nan', 'Vegetarian Friendly'))

In [380]:
data.info()

Создадим новые виды кухонь объединяя, укрупняя и перегруппируя существующие значения. Создадим новый список кухонь.

In [381]:
data['Meat'] = data['Cuisine Style'].apply(lambda x: 1 if ('Grill' or  'Steakhouse' or 'Barbecue') in x else 0)
data['Bar'] = data['Cuisine Style'].apply(lambda x: 1 if ('Bar' or  'Pub' or 'Gastropub' or 'Wine Bar' or 'Brew Pub') in x else 0)
data['Fast_food'] = data['Cuisine Style'].apply(lambda x: 1 if ('Cafe' or  'Fast food' or 'Diner' or 'Street Food' or 'Pizza') in x else 0)
data['Healthy'] = data['Cuisine Style'].apply(lambda x: 1 if ('Vegetarian Friendly' or  'Vegan Options' or 'Gluten Free Options' or 'Soups' or
                                                              'Healthy' or 'Seafood' or 'Sushi') in x else 0)
data['West_European'] = data['Cuisine Style'].apply(lambda x: 1 if ('French' or 'European' or 'Central European' or 'Portuguese' or 'German'
                                                                    or 'Spanish' or 'Austrian' or 'Belgian') in x else 0)
data['East_European'] = data['Cuisine Style'].apply(lambda x: 1 if ('Czech' or 'Eastern European'  or 'Polish' or 'Hungarian') in x else 0)
data['North_European'] = data['Cuisine Style'].apply(lambda x: 1 if ('British' or 'Scandinavian' or 'Dutch' or 'Irish' or 'Danish' or
                                                                     'Swedish') in x else 0)
data['South_European'] = data['Cuisine Style'].apply(lambda x: 1 if ('Mediterranean' or 'Italian' or 'Greek') in x else 0)
data['East'] = data['Cuisine Style'].apply(lambda x: 1 if ('Middle Eastern' or 'Indian' or 'Halal' or 'Turkish' or 'Lebanese') in x else 0)
data['International'] = data['Cuisine Style'].apply(lambda x: 1 if ('International') in x else 0)
data['Asian'] = data['Cuisine Style'].apply(lambda x: 1 if ('Asian' or 'Vietnamese' or 'Chinese' or 'Japanese') in x else 0)
data['American'] = data['Cuisine Style'].apply(lambda x: 1 if ('American' or 'Mexican' or 'Latin' or 'South American') in x else 0)
data['Fashion_Cusine'] = data['Cuisine Style'].apply(lambda x: 1 if ('Contemporary' or 'Delicatessen' or 'Fusion') in x else 0)

In [382]:
data.info()

In [383]:
new_cuisine = pd.DataFrame(['Meat','Bar', 'Fast_food','Healthy','European','West_European','East_European', 'North_European','South_European',
                            'East','International','Asian', 'American','Fashion_Cusine'])

In [384]:
data.info()

5. Обработка признака Ranking. Проведем осмотр признака, проверим признаки, наличие выбросов

In [385]:
column_info(data['Ranking'])

In [386]:
ejection('Ranking')

Незначительное количество значений находится за верхней границей выброса, удаляться эти данные не будут

In [387]:
data['Ranking'].hist(bins=100)

Создадим признак показывающий отношение Ranking к общему числу ресторанов в City

In [388]:
data['Weighted_Ranking'] = data.Ranking / data.Restaurants_Count
data['Weighted_Ranking'].sample(10)

Добавим признак в список числовых

In [389]:
num_cols.append('Ranking')
num_cols.append('Weighted_Ranking')

6. Обработка целевой переменной Rating

In [390]:
column_info(data['Rating'])

In [391]:
data['Rating'].hist(bins=7)

Большинство значений в целевой переменной от 3 до 5. При этом у 10 000 оценок значение 0 что может говорить о том что клиент сделал отзыв о ресторане но не проставил оценку.

7. Обработка признака Price Range. Проведем осмотр показателя

In [392]:
column_info(data['Price Range'])

Примерно 70 процентов ресторанов из рейтинга относятся к средней ценовой категории, примерно 20 процентов относится к низшей ценовой категории и 10 процентов к дорогим ресторанам. Заполним пропущенные значения наиболее часто встречающимся.

In [393]:
data['Price Range'] = data['Price Range'].fillna(data['Price Range'].value_counts().index[0])
column_info(data['Price Range'])

Изменим значения признака на числовые

In [394]:
data['Price Range'] = data['Price Range'].astype(str).apply(lambda x: x.replace("$$ - $$$", '2').replace('$$$$', '3').replace("$", '1'))
column_info(data['Price Range'])

Сгруппируем ценовые диапазоны и рейтинг ресторанов - рейтинг ресторана в среднем слабо зависит от ценовой категории ресторана, его положения в рейтинге. При этом есть слабая обратная корреляция с Restaraunt_ID - дорогих ресторанов меньше чем средней и низкой ценовой категории.

In [395]:
PR = data.groupby('Price Range')['Rating'].mean()
display(PR)

In [396]:
PR1 = data.groupby('Price Range')['Ranking'].mean()
display(PR1)

In [397]:
data.corr()['Rating']

In [398]:
num_cols.append('Price Range')

8. Проведем осмотр признака Number of Reviews.

In [399]:
column_info(data['Number of Reviews'])

In [400]:
ejection('Number of Reviews')

Признак содержит 3200 пропусков, есть выбросы, при этом 75 процентов значений ниже верхней границы выбросов. Выбросы пока удалять не будем, посмотрим на результаты модели с ними и без них.

In [401]:
data['Number of Reviews'].plot()

Построим график для сравнения рейтинга и количества отзывов, проведем корреляцию показателй

In [402]:
plt.figure(figsize=(10, 5))
a = sns.scatterplot(x = 'Number of Reviews', y = 'Rating', data = data, alpha=0.5)
a.set_title('Rating / Number of Reviews')
a.set_ylabel('Rating')
plt.show()

Наибольшим количеством отзывов обладают рестораны с рейтингом от 3,5 до 4,5. Выбросы на графике связаны с популярностью конкретных ресторанов или видов кухни

In [403]:
data.corr()['Number of Reviews']

Есть прямая и обратная корреляция между количеством обзоров и видами кухни: у здоровой кухни и южноевропейской прямая корреляция, а у фастфуда обратная

Заменим пропуски в данных на медиану

In [404]:
data['Number of Reviews'].fillna(data['Number of Reviews'].median(), inplace=True)
column_info(data['Number of Reviews'])

In [405]:
num_cols.append('Number of Reviews')

9. Проведем осмотр признака Reviews.

In [406]:
column_info(data['Reviews'])

В признаке содержится текст последних отзывов с датами, выделим каждый отзыв и дату в отдельные столбцы.

In [407]:
data_reviews = data['Reviews'].str.split('|', expand=True)
data_reviews.columns = ['First_review', 'Second_review', 'First_date', 'Second_date','Na']
data_reviews['Na'].value_counts()
data_reviews[data_reviews.Na == '07/19/2017']
data_reviews.drop(columns=['Na'], inplace=True)
print(data_reviews.info())

Заполним NaN в датах наиболее часто встречающимся значением, проведем обработку дат, переведем их в формат чисел, рассчитаем количество дней, разницу в днях между датами и текущим временем

In [408]:
data_reviews['First_date'] = data_reviews['First_date'].astype(str).apply(lambda x: None if x.strip() == '' else x)
data_reviews['First_date'] = data_reviews['First_date'].apply(lambda x: x.replace('None', '07/01/2018'))
data_reviews['First_date'] = data_reviews['First_date'].astype(str).apply(lambda x: x.replace('Pub/Restaurant', '07/19/2017'))
data['First_date'] = pd.to_datetime(data_reviews['First_date'])
data_reviews['Second_date'] = data_reviews['Second_date'].astype(str).apply(lambda x: None if x.strip() == '' else x) 
data_reviews['Second_date'] = data_reviews['Second_date'].apply(lambda x: x.replace('None', '2018-03-01'))
data['Second_date'] = pd.to_datetime(data_reviews['Second_date'])
data['Days1'] = data['First_date'].dt.day
data['Days2'] = data['Second_date'].dt.day
data['timedelta_1'] = (data['First_date'] - data['Second_date']).astype('timedelta64[D]')
data['current_date1'] = datetime.today()
data['timedelta_2'] = (data['current_date1'] - data['First_date']).astype('timedelta64[D]')
data['timedelta_3'] = (data['current_date1'] - data['Second_date']).astype('timedelta64[D]')
num_cols.append('Days1')
num_cols.append('Days2')
num_cols.append('timedelta_1')
num_cols.append('timedelta_2')
num_cols.append('timedelta_3')

Посмотрим на корреляцию разницы в днях между первм и вторым отзывом количеством дней

In [409]:
data.corr()['timedelta_1']

Сильная прямая корреляция 0.61 между разницей в днях между первым и вторым отзывом и между текущей датой и вторым отзывом

In [410]:
plt.figure(figsize=(10, 5))

g = sns.barplot(x = 'Rating', y = 'timedelta_1', capsize = 0.1, data = data)
g.set_title('Days_between_reviews / Rating')
g.set_ylabel('Days_between_reviews')
plt.show()

Количество дней между первым и вторым отзывами больше всего  у ресторанов с рейтингом 2.5 и меньше всего у 4-4.5. У ресторанов с рейтингом 1.5-2.5 есть выбросы в данных.

Перенесем дату из столбца N в столбец Second_date, удалим лишний столбец Na, символы и цифры из текста отзывов, удалим None и сделаем буквы маленькими

In [411]:
data_reviews['Second_review'] = data_reviews['Second_review'].fillna('')
data_reviews['All_reviews'] = data_reviews['First_review'].str.cat(data_reviews['Second_review'],sep=' ')
data_reviews['All_review'] = data_reviews['All_reviews']
data_reviews['All_review'] = data_reviews['All_review'].apply(clean_symbs)
data_reviews['All_review'] = data_reviews['All_review'].str.lower()
data_reviews['All_review1'] = data_reviews['All_review'].str.strip(' ')
data_reviews['All_review1'].sample(5)

Загрузим лист стоп слов на английском и добавим в него новые значения исходя из наших отзывов

In [412]:
stopwords_en = stopwords.words('english')
newstopwords_en = [' ','food']
stopwords_en.extend(newstopwords_en)
print(stopwords_en)

Лематизируем текст и убираем стоп-слова, посмотрим наиболее часто используемые фразы и скопируем новые данные в основной дата фрейм

In [413]:
data_reviews['All_review2'] = data_reviews['All_review1'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords_en)]))
import nltk
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]
data_reviews['All_review3'] = data_reviews['All_review2'].apply(lemmatize_text)
data['Reviews_Clean'] = data_reviews['All_review3']
MC = data_reviews['All_review3']
counter = Counter()
def count_words(sentence):
    global counter
    for x in sentence:
        counter[x] += 1
      
MC.apply(count_words)
counter.most_common(n=15)

Создадим новые признаки при помощи SentimentIntensityAnalyzer из nltk с помощью следующих действий
1. Переведем списки в текст
2. Применим sia после чего получим Series состоящий из словарей со значениями neg, neutral, pos и compound
3. Создадим датафрейм где признаками будут ключи словарей
4. Соединим датафрейм с основным набором данных

In [414]:
text_raw_str = data['Reviews_Clean'].apply(lambda x: ' '.join(x))

sia = SentimentIntensityAnalyzer()
Sentiment_dicts = text_raw_str.apply(lambda x: sia.polarity_scores(x))

sentiments_df = pd.DataFrame(Sentiment_dicts.values.tolist())

data = pd.concat([data, sentiments_df], axis=1)

Посмотрим на диаграмме как соотносится рейтинг ресторана и результат работы SentimentIntensityAnalyzer - признак compound (cоставной балл представляет собой сумму положительных, отрицательных и нейтральных баллов, которая затем нормализуется между -1 (наиболее отрицательный результат) и +1 (самый крайний положительный результат))

In [415]:
plt.figure(figsize=(10, 5))

g = sns.barplot(x = 'Rating', y = 'compound', data = data)
g.set_title('Compund / Rating')
g.set_ylabel('Compound')
plt.show()

На графике отображается прямая зависимость рейтинга ресторана от отзывов клиентов - чем лучше отзывы тем выше рейтинг и наоборот. Возможно 0 ретинг обладает большим количество положительных отзывов из-за того что клиент оставил положительный отзыв но не поставил значение оценки рейтинга на сайте. У значений рейтинга 1-1.5 есть выбросы.

10. Проведем осмотр признака ID_TA: ID ресторана в TripAdvisor, посмотрим на корреляцию

In [416]:
column_info(data['ID_TA'])

In [417]:
data.corr()['ID_TA'].sort_values()

Пропусков в данных нет. Есть слабая обратная корреляция -0.285681 с Number of Reviews, количеством дней от первого и второго отзыва -0.20 что теоретически может быть связано с моментом появления ресторана на сайте TripAdvisor

In [418]:
num_cols.append('ID_TA')

11. URL_TA: страница ресторана на 'www.tripadvisor.com'

In [419]:
column_info(data['URL_TA'])

Пропусков нет

Проведем корреляцию признаков с целевой переменной и построим тепловую карту

In [420]:
data.corr()['Rating'].sort_values()

In [421]:
fig = plt.figure(figsize=(20,20))
sns.heatmap(data.corr(), annot=True, cmap='RdBu')

In [422]:
data.info()

Создадим dummy переменные для City

In [423]:
City_dummies = pd.get_dummies(data['City'])
data = data.join(City_dummies)

In [424]:
# скопируем датафрейм для работы
data_nom = data.copy()
# запустим функции построения boxplot и поиска статистических различий в распределении оценок с помощью теста Стьюдента
for column in ['Restaurant_id','Ranking', 'Price Range', 'Number of Reviews','Meat', 'Bar', 'Fast_food','Healthy','West_European','East_European', 'North_European','East','Asian','American',
              'Fashion_Cusine','Days1','Days2','timedelta_1','timedelta_2','timedelta_3','neg','neu','pos','compound','City',
              'City_Population','Restaurants_Count','Population_Restaurants','Weighted_Ranking']:
    get_boxplot(data_nom, column, 'Rating')
    get_stat_dif(data_nom, column, 'Rating')

Стандартизируем числовые признаки

In [425]:
scaler = StandardScaler()
scaler.fit(data[num_cols])
data[num_cols] = scaler.transform(data[num_cols])
print(data[num_cols])

# Data Preprocessing
Удаляем признаки, которые не будут использоваться в модели

In [426]:
data.drop(['First_date','Second_date', 'current_date1','Reviews','Reviews_Clean','Cuisine Style','City','URL_TA'], axis=1, inplace=True)

Теперь выделим тестовую часть

In [427]:
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values
X = train_data.drop(['Rating'], axis=1)

Воспользуемся специальной функцие train_test_split для разбивки тестовых данных и выделим 20% данных на валидацию (параметр test_size)

In [428]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

проверяем

In [429]:
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [430]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [431]:
# Создаём модель
model = RandomForestRegressor(n_estimators=100, random_state=RANDOM_SEED)

In [432]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке, предсказанные значения записываем в переменную y_pred

In [433]:
y_pred = model.predict(X_test)

Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.

In [434]:
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [435]:
# Посмотрим на наиболее важные для модели признаки
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(70).plot(kind='barh')

# Submission 

In [436]:
test_data.sample(10)

In [437]:
test_data = test_data.drop(['Rating'], axis=1)

In [438]:
sample_submission

In [439]:
predict_submission = model.predict(test_data)

In [440]:
predict_submission

In [441]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)